Cutting the images

In [ ]:
import pandas as pd
import cv2
import os

folders = ["train","test"]

for folder in folders:
    annotation_path = os.path.join(folder, "_annotations.csv")
    annotations = pd.read_csv(annotation_path)

    for idx, row in annotations.iterrows():
        image_name = row['filename']
        x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

        image_path = os.path.join(folder, image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Image {image_name} in {folder} not found.")
            continue

        cropped_image = image[y_min:y_max, x_min:x_max]

        cv2.imwrite(image_path, cropped_image)

        print(f"Cropped image overwritten at: {image_path}")


Converting files to its corresponding class 

In [ ]:
import os

import pandas as pd

import shutil



folders = ['train','test']



for folder in folders:

    csv_path = os.path.join(folder, '_annotations.csv')

    

    if os.path.exists(csv_path):

        df = pd.read_csv(csv_path)

        

        for index, row in df.iterrows():

            original_filename = row['filename']

            class_label = row['class']

            

            new_filename = f"{class_label}.{index}.jpg" 

            

            original_file_path = os.path.join(folder, original_filename)

            

            class_folder_path = os.path.join(folder, class_label)

            if not os.path.exists(class_folder_path):

                os.makedirs(class_folder_path)

            

            new_file_path = os.path.join(class_folder_path, new_filename)

            

            if os.path.exists(original_file_path):

                shutil.move(original_file_path, new_file_path)

                print(f"Moved {original_filename} to {new_file_path}")

            else:

                print(f"File {original_filename} not found in {folder}!")

    else:

        print(f"CSV file not found in {folder}!")


Reading the data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/hieroglyph3/data/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    '/kaggle/input/hieroglyph3/data/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/hieroglyph3/data/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
import matplotlib.pyplot as plt



def display_sample(dataset, sample_size=9):

    images, labels = next(iter(dataset.take(1)))

    

    plt.figure(figsize=(10, 10))

    

    for i in range(sample_size):

        ax = plt.subplot(3, 3, i + 1)  

        plt.imshow(images[i].numpy().astype("uint8"))  

        plt.title(f"Class: {labels[i].numpy().argmax()}")  

        plt.axis("off") 

    

    plt.show()



display_sample(train_generator)


In [ ]:
display_sample(test_dataset)


# Creating the models

CNN model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, models

l2_reg = keras.regularizers.l2(0.001)


inputs = keras.Input(shape=(224,224,3))
x = layers.Rescaling(1. /255)(inputs)
x = layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
    
x = layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
    
x = layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
    
x = layers.Conv2D(filters = 265, kernel_size = 3, activation = 'relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.3)(x)

x = layers.Conv2D(filters = 512, kernel_size = 3, activation = 'relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(512, activation = 'relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x) 
x = layers.Dropout(0.5)(x)
    
    
outputs =  layers.Dense(95, activation='softmax')(x)
CNNmodel = keras.Model(inputs=inputs,outputs=outputs) 
    
    
CNNmodel.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy','recall','precision']
    )

DenseNet121 model

In [ ]:
base_model = keras.applications.DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))
base_model.trainable = False

inputs = keras.Input(shape=(224,224,3))
x =base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256,activation='relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(95,activation='softmax')(x)
    
dnsmodel = keras.Model(inputs=inputs, outputs=outputs)
    
dnsmodel.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy','recall','precision']
    )
    

ResNet50 model

In [ ]:
    
base_model = keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
)
    
base_model.trainable = False
    
inputs = keras.Input(shape=(224,224,3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256,activation='relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(95,activation='softmax')(x)
    
rsnmodel = keras.Model(inputs=inputs, outputs=outputs)
    
rsnmodel.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy','recall','precision']
)
    
    

EfficientNetB0 model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, models

l2_reg = keras.regularizers.l2(0.001)

base_model = keras.applications.EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
base_model.trainable = False
     
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256,activation='relu',kernel_regularizer=l2_reg)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(95, activation="softmax")(x)
    
efnmodel = keras.Model(inputs, outputs)
    
efnmodel.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy','recall','precision']
)
    



# Fitting the models

Fitting the CNN model

In [ ]:
cnncallbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2, patience=3,min_lr=1e-6),
    keras.callbacks.ModelCheckpoint(monitor='val_loss',filepath='cnnbest_model.keras', save_best_only=True)
]


cnn_history = CNNmodel.fit(train_generator,
                           epochs=30,
                           validation_data=valid_generator,
                           callbacks=cnncallbacks)


In [ ]:
import matplotlib.pyplot as plt
accuracy = cnn_history.history["accuracy"]
val_accuracy = cnn_history.history["val_accuracy"]
loss = cnn_history.history["loss"]
val_loss = cnn_history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

fitting the ResNet50 model

In [ ]:
rsncallbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath='rnsbest_model.keras', save_best_only=True)
]

rsn_history = rsnmodel.fit(train_generator,
                           epochs=20,
                           validation_data=valid_generator,
                           callbacks=rsncallbacks)

In [ ]:
import matplotlib.pyplot as plt
accuracy = rsn_history.history["accuracy"]
val_accuracy = rsn_history.history["val_accuracy"]
loss = rsn_history.history["loss"]
val_loss = rsn_history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Fitting the DenseNet50 model

In [ ]:
dnscallbacks = [ 
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath='dnsbest_model.keras', save_best_only=True)
]

dns_history = dnsmodel.fit(train_generator,
                           epochs=30,
                           validation_data=valid_generator,
                           callbacks=dnscallbacks)

In [ ]:
import matplotlib.pyplot as plt
accuracy = dns_history.history["accuracy"]
val_accuracy = dns_history.history["val_accuracy"]
loss = dns_history.history["loss"]
val_loss = dns_history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Fitting the EfficientNetB0 model

In [ ]:
efncallbacks = [ 
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath='efnbest_model.keras', save_best_only=True)
]

efn_history = efnmodel.fit(train_generator,
                           epochs=30,
                           validation_data=valid_generator,
                           callbacks=efncallbacks)

In [ ]:
import matplotlib.pyplot as plt
accuracy = efn_history.history["accuracy"]
val_accuracy = efn_history.history["val_accuracy"]
loss = efn_history.history["loss"]
val_loss = efn_history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

# Evaluation

In [ ]:
cnn_test_model = keras.models.load_model(
"cnnbest_model.keras")
results = cnn_test_model.evaluate(test_generator)

cnntest_loss = results[0]
cnntest_acc = results[1]
cnntest_recall = results[2]  
cnntest_precision = results[3]

print(f"Test loss: {cnntest_loss:.3f}")
print(f"Test accuracy: {cnntest_acc:.3f}")
print(f"Test recall: {cnntest_recall:.3f}")
print(f"Test precision: {cnntest_precision:.3f}")

In [ ]:
rns_test_model = keras.models.load_model(
"rnsbest_model.keras")
results = rns_test_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.3f}")

rnstest_loss = results[0]
rnstest_acc = results[1]
rnstest_recall = results[2]  
rnstest_precision = results[3]

print(f"Test loss: {rnstest_loss:.3f}")
print(f"Test accuracy: {rnstest_acc:.3f}")
print(f"Test recall: {rnstest_recall:.3f}")
print(f"Test precision: {rnstest_precision:.3f}")

In [ ]:
dns_test_model = keras.models.load_model(
"dnsbest_model.keras")
results = dns_test_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.3f}")

dnstest_loss = results[0]
dnstest_acc = results[1]
dnstest_recall = results[2]  
dnstest_precision = results[3]

print(f"Test loss: {dnstest_loss:.3f}")
print(f"Test accuracy: {dnstest_acc:.3f}")
print(f"Test recall: {dnstest_recall:.3f}")
print(f"Test precision: {dnstest_precision:.3f}")

In [ ]:
efn_test_model = keras.models.load_model(
"efnbest_model.keras")
results = efn_test_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.3f}")

efntest_loss = results[0]
efntest_acc = results[1]
efntest_recall = results[2]  
efntest_precision = results[3]

print(f"Test loss: {efntest_loss:.3f}")
print(f"Test accuracy: {efntest_acc:.3f}")
print(f"Test recall: {efntest_recall:.3f}")
print(f"Test precision: {efntest_precision:.3f}")

In [ ]:
import pandas as pd

data = {
    "Model": ["CNN", "ResNet", "DenseNet", "EfficientNet"],
    
    "Train Accuracy": [0.99, 0.94, 0.99, 0.01],  
    "Train Recall": [0.99, 0.98, 0.99, 0.01],    
    "Train Precision": [0.99, 0.97, 0.99, 0.01], 
    
    "Test Accuracy": [cnntest_acc, rnstest_acc, dnstest_acc,efntest_acc],
    "Test Recall": [cnntest_recall,rnstest_recall, dnstest_recall,efntest_recall],
    "Test Precision": [cnntest_precision, rnstest_precision, dnstest_precision,efntest_precision],
    "Test Loss": [cnntest_loss, rnstest_loss, dnstest_loss, efntest_loss]
}

df_metrics = pd.DataFrame(data)

print(df_metrics)
